<a href="https://colab.research.google.com/github/MatchLab-Imperial/deep-learning-course/blob/master/week09_Deep_Reinforcement_Learning_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Deep Policy Gradient Methods 👾

Welcome to this new DRL tutorial🤘 🤘 !  $\underline{\text{Let's recap a bit}}$: in the previous tutorial we introduced reinforcement learning value-based methods, in particular, we implemented a deep Q-learning approach for the cart-pole problem. Q-learning uses a function approximation to learn the action-value functions, i.e., we have a function $f_{\theta}(x)$ parameterized by  $\theta$ (a neural net in our cart-pole problem), that for the set of different possible actions $\mathcal{A}$, outputs the expected return of tanking these actions $a \in \mathcal{A}$ in state $s$ that is $f_{\theta}\left(s,a\right)\approx Q_{\theta}\left(s,a\right)$, where $Q_{\theta}\left(s,a\right)$ is the expected return of taking action $a$ in state $s$ and following policy $\pi$ afterwards.

Value base methods try to estimate either the 


*   $\underline{\text{State-value functions}}$: $V_{\theta}\left(s\right)\approx V_{\theta}^{\pi}\left(s\right)\ \doteq \mathop{\mathbb{E_\pi}} \left[\sum ^{\infty} _{k=0} \gamma ^t R_{t+k+1} \mid S_t=s\right]$ 

> or

*    $\underline{\text{Action-value functions}}$: $Q_{\theta}\left(s,a\right)\approx Q_{\theta}\left(s,a\right)\doteq \mathop{\mathbb{E_\pi}}\left[\sum ^{\infty} _{k=0} \gamma ^t R_{t+k+1}\mid S_t=s,A_t=a\right]$.



The policy $\pi$ is generated directly from these function estimates, e.g., in Q-learning we select the action with the highest estimated action-values, thus, the policies would not even exist without the action-value estimates. An in order to cover all possible actions, we use an $\epsilon$-greedy policy, then, Q-Learning implies a trade-off between exploration and exploitation.  

In this tutorial we will dive in the world of **deep policy gradient** methods, which instead of learning a value functions estimates, directly learn a parameterized policy $\pi_{\theta}()$ that can select actions without estimating the value functions, so that the probability that action $a$ is taken given that the environment is in state $s$  with parameter $\theta$ is:
\begin{equation}
\pi_{\theta}\left(a\mid s\right)= P\left(a \mid s, \theta\right)
\end{equation}

We will be implementing a policy gradient method in the  [Pong environment](https://gym.openai.com/envs/Pong-v0/).

![alt text](https://flyyufelix.github.io/img/ddqn.gif)

Example of  a policy gradient agent playing [Doom](https://en.wikipedia.org/wiki/Doom_(1993_video_game). 


---



---



# Policy Gradient Methods 🔥 🔥 

Policy gradient methods learn a parameterization of the  policy, $\pi _{\theta}$, and they optimize the parameters $\theta$ by  directly applying [gradient ascent](https://en.wikipedia.org/wiki/Gradient_descent) on a performance objective function $J \left( \theta\right)$. As we learn directly a policy, the probability of taking action $a$ given state $s$ can be denoted by

\begin{equation}
\pi _{\theta}\left( a\mid s\right)= P\left(a \mid s, \theta\right).
\end{equation}

Where $P\left(a \mid s, \theta\right)$ is the probability of selecting action $a$ being in state $s$ with parameters $\theta$. Policy gradient methods output a probability distribution over the action space $\mathcal{A}$ and actions are selected according to this distribution. These methods improve their parameters $\theta$ based on the gradient of some performance  measure $J$ with  respect  to  the  policy  parameter $\theta$ .   We  define $J \left( \theta\right)$ as
\begin{equation}
J \left( \theta\right)=\mathbb{E_{\tau \sim\pi_{\theta}}}\left[\sum_{t=0}\gamma ^{t}r_{t+1}\right]=\sum_{\tau \in \mathcal{D}}p\left(s_0\right)\prod_{t}P\left(S_{t+1}\mid S_t,A_t\right)\pi _{\theta}\left( A_t\mid S_t\right)\mathcal{R}_{s,a},
\end{equation}

corresponding to the expected discounted reward. Where  $D$ is the set of all trajectories observed (remember that a trajectory $\tau$ is a tuple of state $s$, action $a$, reward $r$ and new state observed $s'$ ).  So for each sampled trajectory, we compute the expected return considering the probability of being in initial state $s_0$ and following the parameterized policy $\pi_\theta$ afterwards, where $P\left(s_{t+1}\mid s,a \right)$ is the transition probabilities of the environment, $ \pi _{\theta}\left( a\mid s\right)$ the probability of taking action $a$ given state $s$, and $\mathcal{R}_{s,a}$ the discounted rewards observed. 

To maximize the performance function  $J \left( \theta\right)$  we use gradient descent:


1.   Compute the gradient $\nabla_{\theta}J \left(\theta\right)$:
\begin{equation}
\nabla_{\theta}J \left( \theta\right)=\nabla_{\theta}\mathbb{E_{\tau \sim\pi_{\theta}}}\left[\sum_{t=0}\gamma ^{t}r_{t+1}\right]=\sum_{\tau \in \mathcal{D}}\nabla_{\theta}P\left(\tau\mid \theta\right)R\left(\tau\right)
\end{equation}
2.   Update the parameters:
\begin{equation}
\theta \leftarrow \theta +\alpha \nabla_{\theta}J \left( \pi_{\theta}\right)
\end{equation}

All methods that follow this general schema are called *policy-based gradient methods*.

**Gradient  🚨:**

If we compute the gradient from expression 1, we lose the probability value and thus, we will no longer be able to compute the expectation based on the trajectories observed.  Follow this [trick](https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html) we multiple and divide $P\left(\tau\mid \theta\right)$ and as the derivate of $\frac{\partial \ln f(x)}{\partial x}=1/ f(x)\cdot f'(x)$ we get:

\begin{equation}
\nabla_{\theta}J \left( \theta\right)=\sum_{\tau \in \mathcal{D}}\nabla_{\theta}P\left(\tau\mid \theta\right)R\left(\tau\right)=\sum_{\tau \in \mathcal{D}}\frac{P\left(\tau\mid \theta\right)}{P\left(\tau\mid \theta\right)}\nabla_{\theta}P\left(\tau\mid \theta\right)R\left(\tau\right)=\mathbb{E_{\tau \sim\pi_{\theta}}}\left[ \sum _{t=0}\nabla_{\theta}\ln \pi_{\theta}\left(a\mid s\right)R\left(\tau\right)\right]
\end{equation}


---



---



# Why Policy Gradient Methods? 🤔

*"But dear GTAs, Deep Q Learning worked great! Why bothering to use policy gradient reinforcement learning methods?"*

For the following reasons young padawan:


---



**-Better convergence properties ⚡️**

In general, policy-based methods have better convergence properties compared to value-based methods.

The problem with value-based methods is that they can have a big oscillation while training since the choice of actions may change dramatically for an arbitrarily small change in the estimated action values.

With policy gradient, we just follow the gradient direction and try to find the best parameters, being our policy update smooth at each step and depending of our step-size $\alpha$.

However, as we follow the gradient to find the best parameters, we are not guaranteed to converge to a global maximum (best case) and we might converge on a local maximum.

![alt text](https://cdn-images-1.medium.com/max/800/1*0lYcY5TBSqfNwdu8TduB6g.png)

Image [source](https://cdn-images-1.medium.com/max/800/1*0lYcY5TBSqfNwdu8TduB6g.png)



---



**-Dealing with High dimensional spaces 🎊**

Policy gradients are more effective in high dimensional action spaces, or when using continuous actions.

As we saw in our previous tutorial, Deep Q-learning outputs the expected future return for each possible action at each time step, given the current state.

But what if we have an infinite number of possible of actions? For instance, with a self-driving car, at each state you can have a huge number of actions (turning the wheel at 15°, 17.2°, 19,4°, honk…). We’ll need to output a Q-value for each possible action 😰!

In contrast, with policy-based methods,  we work with the policy directly, as a result, there is no need for computing all the actions estimates at each time step 🙏.

![alt text](https://cdn-images-1.medium.com/max/800/1*_hAkM4RIxjKjKqAYFR_9CQ.png)

Image [source](https://cdn-images-1.medium.com/max/800/1*_hAkM4RIxjKjKqAYFR_9CQ.png)



---






**-Can learn stochastic policies 🔝**


A deterministic policy is a policy that maps unequivocally states to actions. Given a state $s$, DQL will always select the same action $a$ to take (the one leading to the highest expected return), that is, the parameterization is a one to one mapping between states and actions that is why we employ $\epsilon$-greedy policy.


In real life it might not be always the case that given a situation, a particular action, say action A, is the best to take, there might be cases where for example, for a given state, is better to take action A $60\%$ of the time and action B the rest $40\%$ of the time, therefore we need stochastic policies. 

**Stochastic policy methods** output a probability distribution over the action space $\mathcal{A}$ and actions are drawn according to this distribution. We illustrate why it might be better to use Stochastic policy methods by means of an example.

**Example: Treasure hunting**

Suppose that we have an intelligent autonomous [submarine](https://www.youtube.com/watch?v=m2uTFF_3MaA) and its goal is to get the treasure and avoid the coral that will block its 	propellers:

<img src="https://drive.google.com/uc?id=1S-UkJ8lim3cLbWedVV30emcN0SA3Jw7j">

However, when we designed our submarine we only thought of the danger of rocks, so our [UUV](https://en.wikipedia.org/wiki/Unmanned_underwater_vehicle) perceive where the rocks are but not the corals. The initial position of our submarine is any random empty point on the grid and the rocks as well as the outside of our grid are states that cannot be entered (as the submarine knows how to detect them).

The problem: the two light yellow squares are aliased states, because the agent equally perceives the rocks on the north and the end of the grid on the south for both states, as a result, it cannot differentiate between states. Under a value-based RL algorithm, we learn a quasi-deterministic policy (“epsilon greedy strategy”), as a consequence, our submarine can get to a final solution where is maximum $Q\left(s,a\right)$ for each state is given by the red arrows as follows:

<img src="https://drive.google.com/uc?id=1EqY82CHtZ51EYdoZzpbp2HKBcu1NV8Tt">


As we can see in the previous figure, if our submarine reaches the right aliased state our poor UUV will be going right and left until kingdom comes 🤦‍♂️🤦‍♀️. 

On the other hand, an optimal stochastic policy will randomly move left or right in aliased states 💯. As a consequence it will not be stuck and will reach the goal state with probability $p=1$ as $t\rightarrow \infty$. The learnt stochastic policy is:

<img src="https://drive.google.com/uc?id=1Zm9zgl7uXM9pHm4iYl6u95DpMAYadMmq">

As a result  the learnt policy would be:

$\pi(\text{rocks north and end grid south}\mid \text{ left})=0.5$

$\pi(\text{rocks north and end grid south}\mid \text{right})=0.5$



---



---



# Deep Policy gradient 🧠

Time for our lovelies deep neural networks to take the spotlight 💃 

As you might have imaged, as parameterization $\theta$ for our policy gradient method, we will employ DNNs. However, this time instead of obtaining the Q-Values for each action, we will be obtaining the probability of taking each action, that is why we are going to use a [SoftMax](https://en.wikipedia.org/wiki/Softmax_function) layer as output.

<img src="https://drive.google.com/uc?id=16d8fLoeSXHbXc02NhRfgP33Tqz9_awj6">

We will be covering two Deep policy gradient, being the first one:

**Reinforce 🤓**

The Policy gradient that we will be implementing first is called  Monte Carlo Policy Gradient, aka *Reinforce*. For the Pong we will consider episodic environments, i.e., there are terminal states where the game ends. The algorithm is as follows:

1.   Input a differentiable  policy parameterization $\pi_{\theta}\left(a\mid s\right)$.
2. Initialize policy parameter $\theta$
3.  Repeat forever:
> 1.   Generate an episode $S_0,A_{0},R{1},...,S_{T−1},A_{T−1},R_T,$ following $\pi_{\theta}\left(·|·\right)$ 
> 2.   For each step of the episode $t= 0,\dots,T−1$:
> >1. $G\leftarrow$ return from step $t$.
> > 2. Update parameters $\theta$ (via back propagation):
$$\theta \leftarrow \theta + \alpha\gamma^tG\nabla_{\theta}\ln\pi_{\theta}\left(A_t\mid S_t\right) $$



---



---



# 🕹️ Let's play Pong 🕹️

Okay, enough talking! Let's play a bit 😋! In this first part of the tutorial we will implement *Reinforce policy gradient* in the classical Atari [Pong](https://gym.openai.com/envs/Pong-v0/) environment. 

The game of Pong is an excellent example of a simple RL task. For generation Z students, in Pong you use one of the paddles (the other is controlled by a decent AI) and you have to bounce the ball past the other player, similar to tennis (I can’t believe that I am explaining Pong 👵👴...). 


![alt text](https://raw.githubusercontent.com/keon/policy-gradient/master/assets/pg.gif)



On the low level the game works as follows: we receive an image frame (a 210x160x3 byte array (integers from 0 to 255 providing the pixel values for the R, G and B colour channels)) and we get to decide if we want to move the paddle UP or DOWN (i.e. a binary choice). After every single choice the game simulator executes the action. Each action is repeatedly performed for a duration of $k$ frames, where $k$ is uniformly sampled from $\{2,3,4\}$ (to emulate human behavior) and the environment gives us a reward: Either a $+1$  if the ball went past the opponent, a $-1$  if we missed the ball, or 0 otherwise. And of course, our goal is to move the paddle so that we get lots of reward 💰. 

---



---


**-DNN topology:  	📊  **

As we are going to deal with the game frames directly we will be using convolution neuronal nets for image processing, in particular we will be using two 2D Convolutional Layers. These networks will be followed by a flatten layer and a fully-connected layers with ReLu activation. Finally, to get the distribution probability we will have two outputs corresponding to the actions up and down, and a SoftMax function to obtain the probability distribution to be used. 

$\underline{\text{Pre-processing Blocks:}}$

We will be using two pre-processing blocks:
>-The $\underline{\text{Down Scaling pre-process}}$ block does the following:
1.   Crop the image:  as we are not interested in the score trackers.
2.   Reduce images resolution: We take only alternate pixels - basically halves the resolution of the image.
3.   Take only one channel colour.
4.   Convert the image to black and white: We set the background to 0(black) and the paddles and the ball to 1.

> -For RL visual games we are not interested in the frames itself but in the difference between two consecutive frames, that is why we employ the  $\underline{\text{consider difference to last}}$ block utility. This block returns us the pixel difference between two consecutive frames.

The Network architecture is depicted below:

<img src="https://drive.google.com/uc?id=1bbkDos1GNdXDWlSi2gpUuGwYY0BkzIgw">










**Keras Trainning problem:**
Remeber that we are interested in maximazing the perfomance metric $J\left(\theta\right)$

$\nabla_{\theta}J \left( \theta\right)=\mathbb{E_{\tau \sim\pi_{\theta}}}\left[ \sum _{t=0}\nabla_{\theta}\ln \pi_{\theta}\left(a\mid s\right)R\left(\tau\right)\right]$

which by the[ law of large numbers](https://en.wikipedia.org/wiki/Law_of_large_numbers#Strong_law) we can approximate by: 

$$\nabla_{\theta}J \left( \theta\right)\approx\frac{1}{|D|}\sum_{a,s,r \in \mathcal{D}}\sum _{t=0}\nabla_{\theta}\ln \pi_{\theta}\left(a\mid s\right)r$$

Where $|\mathcal{D}|$ is the cardinality of the set $\mathcal{D}$. So the function that we want to calculate the gradient over is:
$$J \left( \theta\right)\approx\frac{1}{|D|}\sum_{a,s,r \in \mathcal{D}}\sum _{t=0}\ln \pi_{\theta}\left(a\mid s\right)r$$

**The problem:** with an automatic differentiation systems like keras, is that we cannot easily set the loss function that must be back-propagated (it is not the case with TensorFlow or Pytorch), in fact we need "True labels" to  compare with. One way to get around this is to design an alternative loss function that has the correct gradient; in our case this is the *[Cross entropy loss](https://rdipietro.github.io/friendly-intro-to-cross-entropy-loss/).*  

When the distribution is over discrete actions, like our example, then the categorical crossentropy can be interpreted as the likelihood. To see this suppose the observed actions $a_i$ and model action a are [one-hot encoded](https://en.wikipedia.org/wiki/One-hot) vectors $a=\left[a_1,a_2…a_M\right]^T$, where M is the number of possible actions. Then,

$$\sum_iH\left(a_i,\pi_{\theta}\left(a\mid s, \theta\right)\right)=-\sum _{m=1}^Ma^m\ln \pi_{\theta}\left(a\mid s\right)$$

See that now we have a minus sign, but thats okay because keras wants to minimize or "Loss function" and we want to maximize it.

---



**Step 0: Import the needed libraries**

We start by importing the needed libraries:
We will be using 3 libraries:
* Keras: for our DNNs.
* OpenAI Gym: for our Pong Environment.

The Reinforce approach does not need *Collection* library as we need to wait until the episode is finished  thus the $\left(S_t, A_t, R_t,S_{t+1}\right)$  transitions will be stored individually on a numpy array.

In [0]:
import gym
import numpy as np
from IPython.display import HTML
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten
from keras.optimizers import Adam
from keras.layers.convolutional import Convolution2D
import os

Using TensorFlow backend.


**Step 1: The Agent**

Let's start by coding a general *policy gradient agent*, as in the previous tutorial we will construct a class for the agent. The state and action size are passed as parameters and we set the discount factor $\gamma=0.99$ and the learning rate $\alpha=1\cdot10^{-3}$.

In [0]:
class PGAgent:
     def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.99
        self.learning_rate = 0.001
        self.states = []
        self.actions = []
        self.rewards = []
        self.model = self._build_model()

Now we define the DDNs architecture; as explained before, we are going to use two CNNs and one fully connected layer of 600. As optimizer we select *adam* optimizer and the loss function, as explained earlier, is the [cross-entropy](https://rdipietro.github.io/friendly-intro-to-cross-entropy-loss/).

In [0]:
     def _build_model(self):
        model = Sequential()
        model.add(Reshape((80,80,1), input_shape=(self.state_size,)))
        model.add(Convolution2D(32, kernel_size=(9, 9), strides=4, activation='relu', init='he_uniform'))
        model.add(Convolution2D(16, kernel_size=(9, 9), strides=2, activation='relu', init='he_uniform'))
        model.add(Flatten())
        model.add(Dense(self.action_size, activation='softmax'))
        opt = Adam(lr=self.learning_rate)
        model.compile(loss='categorical_crossentropy', optimizer=opt)
        return model

Now define the method to store the trajectories, i.e., $S_t,A_t,R_t,S_{t+1}$ tupples into different lists. 

In [0]:
def remember(self, state, action, reward):
        y = np.zeros([self.action_size])
        y[action] = 1
        self.actions.append(y)
        self.states.append(state)
        self.rewards.append(reward)

We select the action. Remember that, now the actions are probability distirbutions, so we use the numpy function [random.choice ](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html) to select one of the possible actions with the probability distribution obtained.

In [0]:
 def act(self, state):
        state = state.reshape([1, state.shape[0]])
        aprob = self.model.predict(state, batch_size=1).flatten()
        action = np.random.choice(self.action_size, 1, p=aprob)[0]
        return action

We compute the discounted reward, where $T$ is the end of our episode:
$\sum_{t=0}^T\gamma ^{t}r_{t}$

In [0]:
def discount_rewards(self, rewards):
        discounted_rewards = np.zeros_like(rewards)
        running_add = 0
        for t in reversed(range(0, rewards.size)):
            if rewards[t] != 0: # if reward!=0 means that we ended the game.
                running_add = 0
            running_add = running_add * self.gamma + rewards[t]
            discounted_rewards[t] = running_add
        return discounted_rewards

Now we define how to train the agent for the crossentropy

*   X = batch of the states that are going to be forward through our DNN
*   Y = $a^m\gamma^{t}R_t$

Such that Keras will try to minimize $$ -\sum _{t=0}^T\sum _{m=1}^Ma^m\gamma^{t}R_t\ln \pi_{\theta}\left(a\mid s\right)$$



In [0]:
def train(self):
        actions = np.vstack(self.actions)
        rewards = np.vstack(self.rewards)
        rewards = self.discount_rewards(rewards)
        rewards = rewards / np.std(rewards - np.mean(rewards))
        actions *= rewards
        X = np.squeeze(np.vstack([self.states]))
        Y = np.squeeze(np.vstack([actions]))
        self.model.train_on_batch(X, Y)
        self.states, self.actions, self.rewards = [], [], []

Code Lab might not recognize all the previous code parts defined under the same class, to this end we run everything this time altogether.

In [0]:
class PGAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.99
        self.learning_rate = 0.001
        self.states = []
        self.actions = []
        self.rewards = []
        self.model = self._build_model()


    def _build_model(self):
        model = Sequential()
        model.add(Reshape((80,80,1), input_shape=(self.state_size,)))
        model.add(Convolution2D(32, kernel_size=(9, 9), strides=4, activation='relu', init='he_uniform'))
        model.add(Convolution2D(16, kernel_size=(9, 9), strides=2, activation='relu', init='he_uniform'))
        model.add(Flatten())
        model.add(Dense(self.action_size, activation='softmax'))
        opt = Adam(lr=self.learning_rate)
        model.compile(loss='categorical_crossentropy', optimizer=opt)
        return model

    def remember(self, state, action, reward):
        y = np.zeros([self.action_size])
        y[action] = 1
        self.actions.append(y)
        self.states.append(state)
        self.rewards.append(reward)

    def act(self, state):
        state = state.reshape([1, state.shape[0]])
        aprob = self.model.predict(state, batch_size=1).flatten()
        action = np.random.choice(self.action_size, 1, p=aprob)[0]
        return action

    def discount_rewards(self, rewards):
        discounted_rewards = np.zeros_like(rewards)
        running_add = 0
        for t in reversed(range(0, rewards.size)):
            running_add = running_add * self.gamma + rewards[t]
            discounted_rewards[t] = running_add
        return discounted_rewards

    def train(self):
        actions = np.vstack(self.actions)
        rewards = np.vstack(self.rewards)
        rewards = self.discount_rewards(rewards)
        rewards = rewards / np.std(rewards - np.mean(rewards))
        actions *= rewards
        X = np.squeeze(np.vstack([self.states]))
        Y = np.squeeze(np.vstack([actions]))
        self.model.train_on_batch(X, Y)
        self.states, self.actions, self.rewards = [], [], []

**Step 4: Preprocessing**

Following we implement:
  *the consider difference to last* block 

*   The *Down Scaling pre-process *
*    The *consider difference to last* block

After these blocks the input image to the net is the following:

![alt text](https://karpathy.github.io/assets/rl/pong.gif)



In [0]:
def preprocess(I):
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(np.float).ravel()

**Step 5: Train the agent**

In [0]:
env = gym.make("Pong-v0")
np.random.seed(0)
state = env.reset()
prev_x = None
score = 0
episode = 0
state_size = 80 * 80
action_size = env.action_space.n
agent = PGAgent(state_size, action_size)

while episode < 10000:
    cur_x = preprocess(state)
    x = cur_x - prev_x if prev_x is not None else np.zeros(state_size)
    prev_x = cur_x

    action = agent.act(x)
    state, reward, done, info = env.step(action)
    score += reward
    agent.remember(x, action, reward)

    if done:
        episode += 1
        agent.train()
        print('Episode: %d - Score: %f.' % (episode, score))
        score = 0
        state = env.reset()
        prev_x = None
        score = 0

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, kernel_size=(9, 9), strides=4, activation="relu", kernel_initializer="he_uniform")`
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, kernel_size=(9, 9), strides=2, activation="relu", kernel_initializer="he_uniform")`


Instructions for updating:
Use tf.cast instead.
Episode: 1 - Score: -20.000000.
Episode: 2 - Score: -21.000000.
Episode: 3 - Score: -21.000000.
Episode: 4 - Score: -21.000000.
Episode: 5 - Score: -20.000000.
Episode: 6 - Score: -19.000000.
Episode: 7 - Score: -20.000000.
Episode: 8 - Score: -20.000000.
Episode: 9 - Score: -21.000000.
Episode: 10 - Score: -20.000000.
Episode: 11 - Score: -21.000000.
Episode: 12 - Score: -18.000000.
Episode: 13 - Score: -21.000000.
Episode: 14 - Score: -21.000000.
Episode: 15 - Score: -21.000000.
Episode: 16 - Score: -20.000000.
Episode: 17 - Score: -21.000000.
Episode: 18 - Score: -20.000000.
Episode: 19 - Score: -19.000000.
Episode: 20 - Score: -20.000000.
Episode: 21 - Score: -21.000000.
Episode: 22 - Score: -20.000000.
Episode: 23 - Score: -20.000000.
Episode: 24 - Score: -21.000000.
Episode: 25 - Score: -21.000000.
Episode: 26 - Score: -21.000000.
Episode: 27 - Score: -20.000000.
Episode: 28 - Score: -20.000000.
Episode: 29 - Score: -21.000000.
Epis

**Be patience  ⌚️**

The previous DNNs trainning might take a while ($\approx 5hrs$)... 

![alt text](https://media.giphy.com/media/tXL4FHPSnVJ0A/giphy.gif)

**Done?**

Okay cool! 🎊 you've finished training the agent, let's have a look of  the result:

In [0]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1

^C


In [0]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()
from gym.wrappers import Monitor
import glob
import io
import base64

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
env = wrap_env(gym.make('Pong-v0'))
state = env.reset()
prev_x = None
score = 0
episode = 0
done = False

while not done:
  screen = env.render()  
  cur_x = preprocess(state)
  x = cur_x - prev_x if prev_x is not None else np.zeros(state_size)
  prev_x = cur_x

  action = agent.act(x)
  state, reward, done, info = env.step(action)
  score += reward

  if done:
      print('Episode: %d - Score: %f.' % (episode, score))
      score = 0     
      score = 0
env.close()
show_video()
state=env.reset()

Episode: 0 - Score: -13.000000.


array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [109, 118,  43],
        [109, 118,  43],
        [109, 118,  43]],

       [[109, 118,  43],
        [109, 118,  43],
        [109, 118,  43],
        ...,
        [109, 118,  43],
        [109, 118,  43],
        [109, 118,  43]],

       [[109, 118,  43],
        [109, 118,  43],
        [109, 118,  43],
        ...,
        [109, 118,  43],
        [109, 118,  43],
        [109, 118,  43]],

       ...,

       [[ 53,  95,  24],
        [ 53,  95,  24],
        [ 53,  95,  24],
        ...,
        [ 53,  95,  24],
        [ 53,  95,  24],
        [ 53,  95,  24]],

       [[ 53,  95,  24],
        [ 53,  95,  24],
        [ 53,  95,  24],
        ...,
        [ 53,  95,  24],
        [ 53,  95,  24],
        [ 53,  95,  24]],

       [[ 53,  95,  24],
        [ 53,  95,  24],
        [ 53,  95,  24],
        ...,
        [ 53,  95,  24],
        [ 53,  95,  24],
        [ 53,  95,  24]]

**But the agent is not that good!! 😐**

I know I know... the agent performance vary way to much between different episodes this is because policy gradient methods have a high variance, as can be seen in the below training curve for our previous implementation:

![alt text](https://raw.githubusercontent.com/keon/policy-gradient/master/assets/score.png)

![alt text](https://media1.giphy.com/media/10jhA2Bo7Xm2OY/giphy.gif?cid=3640f6095c82a6e538777956595f3575)

If we could only combine policy gradient with value-based methods 🤔


---



---



#Actor-critic Method 💖

Until now, we have studied two different reinforcement learning methods:


 
* $ \underline{\text{Value based methods}}$: where we learnt a value function that will map each state action pair to the expected return. 
*  $ \underline{\text{Policy based methods}}$: where we directly optimize the policy without using a value function. 

Now we will learn a third one: The **actor-critic ** method. This method is a combination of the previous methods as it learns both, the action-value functions and a policy.

![alt text](https://drive.google.com/uc?id=1bbSdlwSX3geHFqEO6shrljqa3sLDQkRJ)


In this method we will have the **actor** that controls how our agent behaves, i.e., selects the action that we must take at every time step (policy-based), and the **critic**, which will judge how good the action taken is.
![alt text](https://cdn-images-1.medium.com/max/800/1*e1N-YzQmJt-5KwUkdUvAHg.png)


**Actor Critic: an introduction**

In the Reinforce policy method implemented before we found ourselves in a situation of Monte Carlo, i.e., waiting until the end of episode to calculate the reward. This might suggest the agent to conclude that if we had a high reward at the end of the episode, all actions that we took were good, even if some were not. Therefore, to have an optimal policy, we need a lot of samples which translates into slow learning as we saw.

The Actor Critic model is a better score function. Instead of waiting until the end of the episode as we do in Reinforce, we make an update at each time step, and the new update rule is given by:

$$\theta= \theta + \alpha\nabla_{\theta}\ln\pi_{\theta}\left(A_t\mid S_t\right)Q_w\left(S_t,A_t\right)$$

Because we do an update at each time step, we can’t use the total rewards R(t). Instead, we need to train a Critic model that approximates the value function (remember that value function calculates what is the maximum expected future reward given a state and an action). This value function replaces the reward function in policy gradient that calculates the rewards only at the end of the episode.


---



# **How Actor-Critic works 🏆:**

In the actor-critic method we have two parameterisations, one for the agent $\theta$ and the other one for the critic $w$. In our case our parameterisations will be given by two distinct DNNs.  

The Critic will be performing a task similar to Q-Learning, as it estimates the action-value function, thus the update rule is given by: 
$$w=w+\alpha\left(R_t+\gamma Q_{w}\left(S_{t+1},A_{t+1}\right) - Q_w\left(S_t,A_t\right)\right).$$

On the other hand the actor update rule will be: 
$$\theta= \theta + \alpha\nabla_{\theta}\ln\pi_{\theta}\left(A_t\mid S_t\right)Q_w\left(S_t,A_t\right)$$

So, the Actor-critic algorithm is as follows:

1.   Input a differentiable policy parameterization $\pi_{\theta}\left(a\mid s\right)$.
2. Input a differentiable action-value parameterization $ Q_{w}\left(a\mid s\right)$.
3. Initialize $\theta,w$
3.  Repeat forever:
> 1.  See state $S$ and selection action $A$ according to $\pi_{\theta}\left(a\mid s\right)$ 
> 2.  Obtain Reward $r$.
> 3. Update the critic:
$$w=w+\alpha\left(R_t+\gamma Q_{w}\left(S_{t+1},A_{t+1}\right) - Q_w\left(S_t,A_t\right)\right).$$
> 4. Update the actor:
$$\theta= \theta + \alpha\nabla_{\theta}\ln\pi_{\theta}\left(A_t\mid S_t\right)Q_w\left(S_t,A_t\right)$$


**Advantage function to stabilize learning: 💡**

I was shown in [this article](https://pdfs.semanticscholar.org/6e30/6ecf4f5ce71a62f2504843753addcccf415c.pdf) that advantage functions help to reduce the variance in value-base methods.

The advantage function is defined as:

\begin{equation}
A\left(s,a\right)= Q\left(s,a\right)- V\left(s\right)
\end{equation}
Where $Q\left(s,a\right)$ is the action-value function and $V\left(s\right)$ is the state value-function. This function help us how good an action is $Q\left(s,a\right)$  compared to the average reward obtained in that state V\left(s\right). In other words, this function calculates the extra reward I get if I take this action. 

* If $A\left(s,a\right)> 0$: The taken action was good so our gradient is pushed in that direction.

* If $A\left(s,a\right) < 0$: Our taken action is worse than the average value of that state, so the gradient is pushed in the opposite direction (-).

The problem of implementing this advantage function is that it requires the estimation of two value functions , one for  Q(s,a) and V(s). But we can solve this thing *via* [bootstraping](https://en.wikipedia.org/wiki/Bootstrapping_%28statistics%29)  👏. Remember that

$$Q\left(s,a\right)=r+\gamma V(S_{t+1}),$$
so we can have 

$$A\left(s,a\right)=r+\gamma V(S_{t+1})-V\left(S_t\right).$$

Thus we will only need to estimate the state-value function and we will bootstrap, sweet 🍫 !

Then, the gradient for the actor can be computed as

\begin{equation}
\nabla J_{actor}\approx \frac{1}{|\mathcal{D}|}\sum_{s,a \in \mathcal{D}}\nabla_{\theta}\ln \pi_{\theta}\left(a\mid s\right)A\left(s,a\right).
\end{equation}
While for the critic we will be using MSE.
\begin{equation}
\nabla J_{critic}\approx \nabla_{w}\frac{1}{|\mathcal{D}|}\sum_{s,a,r,s_{t+1} \in \mathcal{D}}\left(V_w\left(s\right)-\left[r+\gamma V\left(s_{t+1}\right)\right]\right)^2
\end{equation}



---



---



# 🕹️ Let's play Pong v2.0 🕹️

Okay now let us get down to brass tacks  and implement the Advantage Actor-Critic (A2C)💣. We will first focus on our DNNs topology.

**-DNN topology📊 : **

We will be adding from the flattern layer a dense layer with one output. This layer will be the one predicting the state-value function $V\left(s\right)$, and the actor will be kept untoched. The new topology is depicted below:

![alt text](https://drive.google.com/uc?id=1dYS0hVwbhvW35rTODGwPhN6i0nlkEgt1)

Let see how the agent is modified compared to the previous version. 

**Step 0: Import the needed libraries**

We start by importing the libraries, for A2C case we will be using Keras [functional model ](https://keras.io/getting-started/functional-api-guide/)and not sequential.


In [0]:
import gym
import numpy as np
from IPython.display import HTML
from keras.models import Model
from keras.layers import Dense, Reshape, Flatten
from keras.optimizers import Adam
from keras.layers.convolutional import Convolution2D
import os

**Step 1: Agent implementation**

We build the model depicted in the above picture.

In [0]:
    def _build_model(self):
        inputs = Input(shape=(self.state_size,), name='input')
        x = Reshape((80, 80, 1), input_shape=(self.state_size,))(inputs)
        x = Convolution2D(32, kernel_size=(9, 9), strides=4, activation='relu', init='he_uniform')(x)
        x = Convolution2D(16, kernel_size=(9, 9), strides=2, activation='relu', init='he_uniform')(x)
        x = Flatten()(x)
        prob = Dense(self.action_size, activation='softmax', name='prob')(x)
        state = Dense(1, name='state')(x)
        model = Model(inputs=inputs, outputs=[prob, state])
        opt = Adam(lr=self.learning_rate)
        model.compile(loss={'prob': 'categorical_crossentropy', 'state': 'mean_squared_error'}, optimizer=opt)
        return model

The act function does not change as the actor optimize the policy directly. Thus:

In [0]:
 def act(self, state):
        state = state.reshape([1, state.shape[0]])
        aprob = self.model.predict(state, batch_size=1)[0].flatten()
        action = np.random.choice(self.action_size, 1, p=aprob)[0]
        return action

The **Act function**, the most important function, we need to compute two gradinets:

\begin{equation}
\nabla J_{actor}\approx \frac{1}{|\mathcal{D}|}\sum_{s,a \in \mathcal{D}}\nabla_{\theta}\ln \pi_{\theta}\left(a\mid s\right)A\left(s,a\right).
\end{equation}

\begin{equation}
\nabla J_{critic}\approx \nabla_{w}\frac{1}{|\mathcal{D}|}\sum_{s,a,r,s_{t+1} \in \mathcal{D}}\left(V_w\left(s\right)-\left[r+\gamma V\left(s_{t+1}\right)\right]\right)^2
\end{equation}

and remember that with A2C we will be training after every interaction with the environment, so batch_size=1.

In [0]:
   def train(self, state, action, reward, next_state, done):
            target = np.array(reward).astype('float32').reshape((1,1))
            next_state = next_state.reshape([1, next_state.shape[0]])
            state = state.reshape([1, state.shape[0]])
            if not done:
                target = reward + self.gamma * self.model.predict(next_state, batch_size=1)[1]
            advantage = target - self.model.predict(state, batch_size=1)[1]
            labels = action*advantage
            self.model.fit({'input': state}, {'prob': labels.reshape(1,self.action_size), 'state': target}, batch_size=1, epochs=1, verbose=0)


Let us again define the agent altogether:

In [0]:
class PGAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        inputs = Input(shape=(self.state_size,), name='input')
        x = Reshape((80, 80, 1), input_shape=(self.state_size,))(inputs)
        x = Convolution2D(32, kernel_size=(9, 9), strides=4, activation='relu', init='he_uniform')(x)
        x = Convolution2D(16, kernel_size=(9, 9), strides=2, activation='relu', init='he_uniform')(x)
        x = Flatten()(x)
        prob = Dense(self.action_size, activation='softmax', name='prob')(x)
        state = Dense(1, name='state')(x)
        model = Model(inputs=inputs, outputs=[prob, state])
        opt = Adam(lr=self.learning_rate)
        model.compile(loss={'prob': 'categorical_crossentropy', 'state': 'mean_squared_error'}, optimizer=opt)
        return model

    def act(self, state):
        state = state.reshape([1, state.shape[0]])
        aprob = self.model.predict(state, batch_size=1)[0].flatten()
        action = np.random.choice(self.action_size, 1, p=aprob)[0]
        return action

    def train(self, state, action, reward, next_state, done):
            target = np.array(reward).astype('float32').reshape((1,1))
            next_state = next_state.reshape([1, next_state.shape[0]])
            state = state.reshape([1, state.shape[0]])
            if not done:
                target = reward + self.gamma * self.model.predict(next_state, batch_size=1)[1]
            advantage = target - self.model.predict(state, batch_size=1)[1]
            labels = action*advantage
            self.model.fit({'input': state}, {'prob': labels.reshape(1,self.action_size), 'state': target}, batch_size=1, epochs=1, verbose=0)


**Step 2: Train the agent**

In [0]:
if __name__ == "__main__":
    env = gym.make("Pong-v0")
    np.random.seed(0)
    state = env.reset()
    prev_x = None
    score = 0
    episode = 0
    state_size = 80 * 80
    action_size = env.action_space.n
    agent = PGAgent(state_size, action_size)
    while episode < 10000:
        cur_x = preprocess(state)
        x = cur_x - prev_x if prev_x is not None else np.zeros(state_size)
        prev_x = cur_x

        action = agent.act(x)
        next_state, reward, done, info = env.step(action)
        score += reward
        action_oh = np.zeros(action_size) # Action one-hot encoding
        action_oh[action] = 1
        next_x = preprocess(next_state)
        next_x = next_x-cur_x
        if done:
            next_x = np.zeros(state_size) - cur_x
        agent.train(x, action_oh, reward, next_x, done)
        state = next_state
        if done:
            episode += 1
            print('Episode: %d - Score: %f.' % (episode, score))
            score = 0
            state = env.reset()
            prev_x = None
            score = 0

**The A2C performance**

In [0]:
env = wrap_env(gym.make('Pong-v0'))
state = env.reset()
prev_x = None
score = 0
episode = 0
done = False

while not done:
  screen = env.render()  
  cur_x = preprocess(state)
  x = cur_x - prev_x if prev_x is not None else np.zeros(state_size)
  prev_x = cur_x

  action = agent.act(x)
  state, reward, done, info = env.step(action)
  score += reward

  if done:
      print('Episode: %d - Score: %f.' % (episode, score))
      score = 0     
      score = 0
env.close()
show_video()
state=env.reset()

**Learning Curve**

The A2C learning curve is depicted below. As we can see the A2C curve is smoother than the Reinforce one, that is thanks to the Critic assessing  the quality of the actions taken by the Actor.

![alt text](https://raw.githubusercontent.com/Alexander-H-Liu/Policy-Gradient-and-Actor-Critic-Keras/master/fig/learning_curve.jpg)

# That is the end of our DRL tutorials 😭

![alt text](https://media2.giphy.com/media/lD76yTC5zxZPG/giphy.gif?cid=3640f6095c878f7b4b354373677fcee2)


If your learning appetite has not been satisfied try to implement the A2C in the [Doom environment](http://vizdoom.cs.put.edu.pl/).

You can also have a look at more sophisticated policy gradient algorithms like [TRPO](https://spinningup.openai.com/en/latest/algorithms/trpo.html) or [PPO](https://spinningup.openai.com/en/latest/algorithms/ppo.html).
